In [1]:
from fyp.db import User, UserInteractorRelationships
from tqdm import tqdm


In [2]:
# Final graph idea:
# gt or et 2 comments total
# at least 2 comments to one user
# connected to at least two users


In [3]:
users = [(user.id, user.twitter_user_id, user.seed, user.hop) for user in tqdm(User.select())]
relationships = [(relationship.id, relationship.user_id, relationship.interactor_id, relationship.count) for relationship in tqdm(UserInteractorRelationships.select())]


100%|██████████| 193557/193557 [00:00<00:00, 580446.95it/s]


In [4]:
totals = {}
relationship_count = {}

for relationship in relationships:
    interactor = relationship[2]
    count = relationship[3]
    
    if interactor not in totals:
        totals[interactor] = count
        relationship_count[interactor] = 1
    else:
        totals[interactor] += count
        relationship_count[interactor] += 1


In [5]:
users_10_replies_or_more = {}

for user_id, total in totals.items():
    if total >= 2:
        users_10_replies_or_more[user_id] = False


In [6]:
len(users_10_replies_or_more)

35787

In [7]:
users_more_than_two_replies_to_one_user = []

for relationship in relationships:
    interactor = relationship[2]
    count = relationship[3]
    if interactor in users_10_replies_or_more and users_10_replies_or_more[interactor] is False and count >= 2:
        users_10_replies_or_more[interactor] = True
        users_more_than_two_replies_to_one_user.append(interactor)


In [8]:
len(users_more_than_two_replies_to_one_user)


29515

In [9]:
final_users_to_keep = [i for i in range(1,31)]

for user in users_more_than_two_replies_to_one_user:
    if relationship_count[user] > 1:
        final_users_to_keep.append(user)


In [10]:
len(final_users_to_keep)


14769

In [11]:
relationships_to_keep_with_nulls = [relationship[0] if relationship[2] in final_users_to_keep else None for relationship in relationships]


In [12]:
relationships_to_keep = []
for relationship in relationships_to_keep_with_nulls:
    if relationship is not None:
        relationships_to_keep.append(relationship)


In [13]:
users_to_delete = User.delete().where(~(User.id << final_users_to_keep))
users_to_delete.execute()


62676

In [14]:
relations_to_delete = UserInteractorRelationships.delete().where(~(UserInteractorRelationships.id << relationships_to_keep))
relations_to_delete.execute()


72529

In [15]:
User.select().where(User.hop == 2).count()


9863